# Классификация обращений граждан

## Загрузка библиотек и подготовка данных

In [1]:
import torch
import pandas as pd
import re
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.svm import LinearSVC
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsRestClassifier
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from simpletransformers.language_representation import RepresentationModel
from scipy.special import softmax
import logging
import warnings
warnings.filterwarnings("ignore")

In [2]:
test = pd.read_csv("test_dataset_test.csv")
df = pd.read_csv("train_dataset_train.csv")

Создадим функцию для очистки одного обращения. В обращении будем оставлять только русские буквы, приводить будем к нижнему регистру, удлять всю пунктуацию.

Также будем отбрасывать некоторые однобуквенные и двухбуквенные "мусорные" остатки фраз. Ряд сокращений заменим на полные слова.

In [3]:
def clear_text(text):
    
    text = " ".join((re.sub(r'[^а-яА-ЯёЁ]', ' ', text).split())).lower()
    
    stop_phrases = []
    garbage_replacement = [" " for phrase in stop_phrases]
    
    for from_, to in zip(stop_phrases, garbage_replacement):
        text = re.sub(from_, to, text, flags=re.IGNORECASE)
        
    stop_words = ['д', 'б', 'г', 'x', 'вк', 'го']
    
    text = " ".join([word for word in text.split() if word not in stop_words])
    text = " ".join(['улица' if word == 'ул' else word for word in text.split() if word not in stop_words])
    text = " ".join(['управляющая компания' if word == 'ук' else word for word in text.split() if word not in stop_words])
    text = " ".join(['торговый центр' if word == 'тц' else word for word in text.split() if word not in stop_words])
    text = " ".join(['многоквартирный дом' if word in ['мкд'] else word for word in text.split() if word not in stop_words])
    text = " ".join(['садоводческое некоммерческое товарищество' if word in ['снт'] else word for word in text.split() if word not in stop_words])
    
    return text

Пример очистки текста

In [4]:
test['Текст Сообщения'][0]

'<p>Здравствуйте. На улице Мира &nbsp;было заменено наружное освещение, а именно заменены лампы на &nbsp;энергосберегающие лампы. На протяжении нескольких месяцев освещение улицы отсутствует. Последний раз когда улица была освещена это зима приблизительно до 23:00 да и то не каждый день. А ведь люди работают в 12 часовую смену и многие возвращаются очень поздно. Данная проблема на всех улицах поселка.&nbsp;</p>'

In [5]:
clear_text(test['Текст Сообщения'][0])

'здравствуйте на улице мира было заменено наружное освещение а именно заменены лампы на энергосберегающие лампы на протяжении нескольких месяцев освещение улицы отсутствует последний раз когда улица была освещена это зима приблизительно до да и то не каждый день а ведь люди работают в часовую смену и многие возвращаются очень поздно данная проблема на всех улицах поселка'

Очистим текст сообщений, тематики и ответственного лица в обучающей выборке

In [6]:
df['text'] = df['Текст Сообщения'].apply(clear_text)
df['Тематика'] = df['Тематика'].apply(clear_text)
df['Ответственное лицо'] = df['Ответственное лицо'].apply(clear_text)
df.head(4)

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория,text
0,2246,Помогите начальник Льговского рэс не реагирует...,нарушения связанные с содержанием электросети ...,администрация льговского района,3,помогите начальник льговского рэс не реагирует...
1,380,<p>По фасаду дома по адресу ул. Урицкого 22 пр...,аварийные деревья,администрация города курска,3,по фасаду дома по адресу улица урицкого проход...
2,2240,Агресивные собаки. На радуге там стая из подро...,безнадзорные животные,администрация города курска,1,агресивные собаки на радуге там стая из подрос...
3,596,<p>На пересечении &nbsp;улиц Сосновская и Бере...,нескошенная сорная растительность в местах общ...,комитет дорожного хозяйства города курска,3,на пересечении улиц сосновская и береговая зав...


Из тестовой выборки оставим только "Текст Сообщения", потому что для предсказания нам запрещено использовать какие-либо колонки кроме текста.

In [7]:
test = test[['Текст Сообщения']]
test['text'] = test['Текст Сообщения'].apply(clear_text)
test.head(4)

,Текст Сообщения,text
0,<p>Здравствуйте. На улице Мира &nbsp;было заме...,здравствуйте на улице мира было заменено наруж...
1,<p>Уже вторую неделю не горит уличное освещени...,уже вторую неделю не горит уличное освещение
2,Не работает освещение во дворе дома 11а по Эне...,не работает освещение во дворе дома а по энерг...
3,После покоса сорной растительности на газоне м...,после покоса сорной растительности на газоне м...


# Заполнение редких категорий по ключевым словам

Посмотрим на количество обращений в каждой категории.

In [8]:
df['Категория'].value_counts()

3     954
0     478
16    149
8     139
4     108
10     48
7      27
1      25
11     19
5      12
13     11
6      10
15      7
9       5
14      4
2       3
12      1
Name: Категория, dtype: int64

Видно, что всего 17 категорий, причем существенная часть из них - очень редкие. На редких примерах сложно будет обучать модели, и они будут лишь приносить шум. Поэтому попробуем подобрать для редких категорий ключевые слова, и найти в тестовой выборке такие обращения, которые содержат подобранные нами ключевые слова

Рассмотрим категорию 12

In [9]:
df[df['Категория'] == 12]

,id,Текст Сообщения,Тематика,Ответственное лицо,Категория,text
652,1576,<p>Не могу получить сертификат</p>,тестовая категория,ответственный пми,12,не могу получить сертификат


С учетом того, что фраза всего одна, и ее тематика - "тестовая категория", то можно предположить, что проходило какое-либо тестирование платформы. И такой случай редкий, поэтому просто отбросим эту категорию.

Рассмотрим категорию 2

In [10]:
df[df['Категория'] == 2]['text'].to_list()

['здравствуйт не могу поставить ребёнка на очередь в детский сад через госуслуги указано что данная услуга недоступна в нашем регионе хотя за две недели до моей последней попытки июня подать можно было когда будет урегулирован это вопрос и люди смогут записать ребёнка на очередь',
 'добрый день администратор сообщества блокирует участников без объяснения причины',
 'доброе утро помогите разобраться в проблеме в личном кабитете госуслуг хочу подать заявку на онлайн голосование к сожалению появляется ошибка данные вашей учётной записи не сопоставлены с данными содержащимися в регистре избирателей участников референдума цик россии данные в личном кабинете госуслуг сверил все верно']

Здесь всего 3 образца обращений, одно из которых объединяет и проблему с госуслугами, и очередь в детский сад, второе - очень размытое, третье - сокрее всего слишком частное.

Поэтому из-за редкости категории, также лучшее ее просто отбросить

Далее рассмотрим категорию 14

In [11]:
df[df['Категория'] == 14]['text'].to_list()

['муп льговское не платят зарплату уже больше двух месяцев муж работает за спасибо которое спасибо тоже не получает руководитель горит зарплату получите не раньше апреля сколько можно терпеть безответственность данной организации по просьбе знакомой пишу',
 'добрый день июля года я был направлен на прохождения обучения и получения дополнительного образования по направлению органов службы занятости населения в гаоу дпо курской области курский областной центр подготовки и переподготовки кадров жкх на оператора котельной от даты постановки на учёт по безработице и до июля я активно искал работу у меня трое детей и прожить на одно пособие просто не возможно июля года я официально трудоустроился в соответствии с законом рф при направлении безработного гражданина на профессиональное обучение он считается занятым в случае если в этом периоде безработный гражданин самостоятельно трудоустроился но продолжает профессиональное обучение регулярно посещает занятия выполняет объем учебной программы 

Здесь 2 человека жалуются на обучение от центра занятости, и еще 2 челоека - на задержки зарплаты в гос.учреждении (что может быть очень широкой категорией)

Выделим обучение от центра занятости как ключевые слова и найдем что-то подходящее в тестовой выборке.

In [12]:
cat_14 = list(test[test['text'].str.contains(r'^(?=.*обучен)(?=.*занято)')].index)
cat_14

[611]

Нашлась 1 фраза, сохраним ее индекс.

Также в одном из обращений было упоминание коррупции, попробуем выделить ключевое слово.

In [13]:
cat_14.extend(test[test['text'].str.contains(r'^(?=.*коррупц)')].index.to_list())
len(cat_14)

3

Добавилось еще 2 фразы

Создадим словарь, где ключами будут индексы обращений в тестовой выборке, а значениями - категории. 

In [14]:
hand_cats = {}
for idx in cat_14:
    hand_cats[idx] = 14

Далее рассмотрим категорию 9

In [15]:
df[df['Категория'] == 9]['text'].to_list()

['здравствуйте все мы пишем и говорим только об одном дороги больницы плохая власть и так далее слежу уже пару лет за сообществами курска но не разу не видел чтобы кто то писал и жаловался на операторов сотовой связи по городам курской области вопрос в обслуживании сотовой связи практически нет но можно было бы улучшить качество обслуживания все же на дворе век проблема появляется на выездах из города а именно на трассах области многие друзья сталкиваются с такой проблемой дозвониться возможно только на номер и все почему так происходит говорить о каких то конкретных участках дорог не будем везде есть такая проблема возьмем например дороги такие как курск железногорск курск обоянь курск щигры курск льгов рыльск курск суджа выезжая из курска в любом направлении мы сталкиваемся с проблемой связи любого оператора нет сети проверено на личном опыте когда едем например в направлении москва санкт петербург сеть в есть везде и всегда курская область пока делает дороги ремонтирует больницы и т

Тут заметно, что люди жалуются на проблемы со связью - в частности с интернетом (мобильным)

Попробуем найти фразы с такими ключевыми словами

In [16]:
cat_9 = test[test['text'].str.contains(r'^(?=.*интерне)(?=.*сотов)')].index.to_list()
cat_9

[707]

In [17]:
for idx in cat_9:
    hand_cats[idx] = 9

Нашли одну, добавили ее индекс в словарь.

Рассмотри категорию 15

In [18]:
df[df['Категория'] == 15]['text'].to_list()

['добрый день прошу обратить внимание на неудовлетворительное состояние спортивной площадки школы курска планируется ли реконструкция стадиона в рамках реализации национальных проектов если да то в какие сроки',
 'отсутствие полной поддержки любительскому хоккею мы сейчас находимся в сочи представляем курская область в ночной хоккейной лиги и никакой поддержки со стороны области нету игроки приехали в чем было стыдно просто смотреть как игры команды со всей россии одеты и как их представляют а мы простите как бомжи',
 'здравствуйте у нас в курск есть замечательный парк железнодорожников но кроме памятников там ничего нет хотелось что бы в данном парке появились спортивные тренажёры и детская площадка с одной стороны парка спортивные тренажёры в с другой стороны парка детская площадка',
 'здравствуйте во льгове процветает коррупция по всем направлениям вот например на протяжении около лет в фоке физкультурно оздоровительном комплексе процветает коррупционная схема в бюджетных деньгах в 

Здесь видно, что люди жалуются на детские площадки, плокие конструкции на них и плохое обустройство парков.

Попробуем выделить ключевые слова

In [19]:
cat_15 = test[test['text'].str.contains(r'^(?=.*детск)(?=.*площ)(?=.*установ)|(?=.*детск)(?=.*площ)(?=.*качел)|(?=.*детск)(?=.*площ)(?=.*парк)')].index.to_list()
len(cat_15)

16

In [20]:
for idx in cat_15:
    hand_cats[idx] = 15

Всего нашли 16 таких обращений и добавили их индексы в словарь.

Далее рассмотрим категорию 6.

In [21]:
df[df['Категория'] == 6]['text'].to_list()

['прошу включить меня и мою семью в очередь на получение материальной помощи',
 'прошу включить меня в очередь на получение материальной помощи как малоимущей семье или в очередь на участие в социальном контракте',
 'посёлок находится в красной зоне в школе очень много заболевших дети чьи родственники больны коронавирусом продолжают посещать занятия больных с поражением лёгких отказываются госпитализировать и больных с каждым днём всё больше',
 'добрый вечер как можно записаться на соц контракт',
 'обращение собственников торговых помещений торгового центра олимпийский в связи с изданием постановлением губернатора курской области от года пг о режиме нерабочих дней в курской области в период с октября года по ноября года',
 'добрый день роман владимирович хочу обратиться к вам с таким вопросом почему вы и ваши коллеги считаете продление нерабочих дней целесообразным то что дети не ходят в сад и школу не работают торговые центры так родители этих детей ходят в продуктовые магазины пользу

Видно, что люди здесь обращались за материальной помощью, аппелировали к тому, что имеют многодетные и малообеспеченные семьи.

Попробуем подобрать такие ключевые слова.

In [22]:
cat_6 = test[test['text'].str.contains(r'^(?=.*матер)(?=.*помощ)|(?=.*малообеспечен)(?=.*семь)|(?=.*многодетн)(?=.*сем)')].index.to_list()
len(cat_6)

10

In [23]:
for idx in cat_6:
    hand_cats[idx] = 6

Всего 10 таких фраз, добавили их в словарь.

Рассмотрим категорию 13

In [24]:
df[df['Категория'] == 13]['text'].to_list()

['предлагаю провести проверку качества данного хлеба и соблюдения технологии производства в целом на заводе в хлебе обнаружена мышиная какашка',
 'на перекрестке улица сумской и дейнеки у сквера с памятником дейнеки на тротуаре и газоне более недель с припаркованных машин и со столов торгуют фруктами и овощами заняли половину тротуара и затоптали газон напротив после торговли оставляют мусор и свои подсобные предметы торговли примите меры',
 'прошу помощи в защите прав жителей домов и улицы гагарина города курска дело в том что в бывшем торговом комплексе курские зори расположенном между указанными домами открылся магазина магнит и построена металлическая разгрузочная площадка во дворе этих домов работы по разгрузке товаров после открытия магазина магнит мешают жителям у которых окна выходят во двор обустройство разгрузочной площадки во дворе наших домов нарушает законодательство российский федерации основным документом который регламентирует деятельность торговых точек в жилой зоне яв

Здесь уже становится заметно, что категорию все труднее обработать несоклькими ключевыми словами - присутствуют темы и про некачественные продукты, и про эпидобстановку, и про банкомат.

Поэтому лучшее ее пропустить и применять для классификации модели машинного обучения

Рассмотрим категорию 5.

In [25]:
df[df['Категория'] == 5]['text'].to_list()

['прошу вас привести в порядок тротуар в доль школьного забора на улица и франко тротуар находится в не удовлетворительно состоянии данный тротуар ремонтировался ещё при ссср после распада ссср тротуар не ремонтировался неужели так сложно положить новый асфальт и установить хотя бы скамейку с урной что бы пенсионеры отдыхали',
 'прошу вас провести онлайн игру послещенную новому году и рождеству было бы замечательно что бы студенты учебных заведений в данной игре учавствовали а в качестве подарков можно было бы сделать складкие подарки с логотипом партии единая россия и сертификат',
 'прошу вас направить письмо руководству сах завода что бы они замени флаг рф',
 'дом книги объект культурного наследия буква о скоро упадёт на голову людям проведите срочные противоаварийные работы барельефа',
 'памятник свиридову имеет повреждения постамента отваливается облицовочная плитка почему такое ужасное качество работ это не допустимо тем более в центре города',
 'уважаемый роман владимирович к вам

Здесь сразу видно, что люди жалуются на состояние памятников, хвалят новый год и призывают повесить баннеры.

Темы узкие, поэтому попробуем подобрать ключевые слова.

In [26]:
cat_5 = test[test['text'].str.contains(r'^(?=.*памятник)|новогодн|музык|постаме|баннер')].index.to_list()
len(cat_5)

12

In [27]:
for idx in cat_5:
    hand_cats[idx] = 5
len(hand_cats.keys())

41

Итак, мы сделали словарь из 41 фразы, для которых вручную проставим категорию в зависимости от ключевых слов. Но это будет сделано после того, как мы спрогнозируем для тестовой выборки принадлежность к категории при помощи моделей машинного обучения.

Также отросим все редкие категории для которых будем проставлять категорию из словаря, они не будут присутствовать в обучающей выборке.

In [28]:
df = df[~(df['Категория'].isin([12, 2, 14, 9, 15, 6, 5]))]

## Выделим выборки

Сделаем обучающую и валидационную выборку, размером 10% от обучающей.

In [29]:
y = df['Категория']
X_train, X_val, y_train, y_val = train_test_split(df, y, test_size=0.1, random_state=42, stratify=y)

Поскольку у нас все еще очень мало примеров для определенных категорий, будем делать процедуру апсэмплинга редких категорий - будем доводить количество примеров до более высокого количества, беря пример несколько раз в обучающей выборке.

In [30]:
def upsample(df, ts, smooth, random_state=2):
    
    data = df.copy()
    
    top = data['Категория'].value_counts(normalize=True)[data['Категория'].value_counts(normalize=True) > ts].index
    
    multipliers = []
    ceil = data['Категория'].value_counts().iloc[0]
    for idx, n in zip(data['Категория'].value_counts().index, data['Категория'].value_counts()):
        if idx not in top:
            multiplier = round(ceil / (smooth * n))
            
        else:
            multiplier = 1

        multipliers.append(multiplier)

        upsampled = pd.DataFrame()

        for target_type, multiplier in zip(data['Категория'].value_counts().index, multipliers):
            upsampled = pd.concat([upsampled] + [data.loc[data['Категория'] == target_type, :]] * multiplier)
            
    upsampled = shuffle(upsampled, random_state=random_state).reset_index(drop=True)
            
    return upsampled

Итак, увеличим количество обращений из редких категорий

In [31]:
upsed = upsample(X_train, 0.03, 5)
y_train = upsed['Категория']
upsed['Категория'].value_counts()

3     859
0     430
10    172
11    170
13    170
7     168
1     161
16    134
8     125
4      97
Name: Категория, dtype: int64

Полученную обучающую выборку будем использовать для векторизации обращений.

Возьмем открытый предобученный русскоязычный энкодер типа Bert - "cointegrated/rubert-tiny2" с huggingface
https://huggingface.co/cointegrated/rubert-tiny2

Преимуществом этой модели являются ее маленький размер и очень высокая скорость работы.

И при помощи него будем получать векторы эмбеддинги для каждого обращения.

Загрузим модель и токенизатор

In [32]:
# # "sberbank-ai/sbert_large_mt_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Напишем функцию для векторизации. В ней оставим возможность для векторизации по двум типам - взять CLS-вектор из начала каждой реплики или усредненнный вектор со всех слоев.

In [33]:
def embed(data, cls=True, length=128, test=False):
    
    if test == 'test':
        col = 1
    elif test == 'cv':
        col = 0
    else:
        col = 5
    
    emb = np.zeros([1, 312])
    batch_size = 128

    for index in range(0, len(data), batch_size):

        batch = list(data.iloc[index: min(index + batch_size, data.shape[0]), col])
        
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=length)

        with torch.no_grad():
            model_output = model(**{k: v.to(model.device) for k, v in encoded_input.items()})
        if cls:
            embeddings = model_output.last_hidden_state[:, 0, :]
        else:
            embeddings = model_output.pooler_output
            
        embeddings = torch.nn.functional.normalize(embeddings)
        emb = np.vstack((emb, embeddings))

    return emb[1:]

Векторизуем обучающую и валидационную выборку.

Будем брать усредненный вектор со всех слоев

In [34]:
%%time
train = pd.DataFrame(embed(upsed, test=False, cls=False))

CPU times: user 1min 5s, sys: 12.6 s, total: 1min 17s
Wall time: 21.7 s


In [35]:
%%time
embs_val = pd.DataFrame(embed(X_val, test=False, cls=False))

CPU times: user 4.91 s, sys: 745 ms, total: 5.66 s
Wall time: 1.49 s


# Обучение моделей

## bert_embeddings + logisticRegression

Сначала обучим простую модель логистической регрессии на векторах-эмбеддингах, которые мы извлекли.

In [36]:
logit = LogisticRegression(random_state=0, penalty='l2', C=1, class_weight='balanced', 
                         max_iter=200)
logit.fit(train, y_train)

LogisticRegression(C=1, class_weight='balanced', max_iter=200, random_state=0)

Проверим целевую метрику на валидационной выборке.

In [37]:
holdout = embs_val

holdout_probs = pd.DataFrame(logit.predict_proba(holdout), columns=logit.classes_)
h_i = holdout_probs.idxmax(axis=1)
        
for col in holdout_probs.columns:
    holdout_probs[col].values[:] = 0
            
for row, idx in zip(holdout_probs.iterrows(), h_i):
    row[1][idx] = 1
    
roc_auc_score(y_val, holdout_probs, multi_class='ovo', labels=logit.classes_)

0.7561862357914988

Значение целевой метрики составило около 0.76, при этом мы отбросили редкие категории

## fine_tuning cointegrated/rubert-tiny2 при помощи библиотеки simple_transformers

Далее попбробуем дообучить рассмотренный энкодер на нашей обучающей выборке.

Нам нужно подготовить обучающую и валидационную выборку в формате text-labels

In [38]:
train_data = upsed[['text', 'Категория']]
train_df = pd.DataFrame(train_data)
train_df.columns = ["text", "labels"]

eval_data = X_val[['text', 'Категория']]
eval_df = pd.DataFrame(eval_data)
eval_df.columns = ["text", "labels"]

Категории должны идти строго по порядку, поэтому перекодируем наши категории

In [39]:
cut_cat = {k: v for k, v in zip(df['Категория'].value_counts().sort_index().index, 
                                range(len(df['Категория'].value_counts())))}

In [40]:
train_df['labels'] = train_df['labels'].map(cut_cat)
eval_df['labels'] = eval_df['labels'].map(cut_cat)
train_df['labels'].value_counts()

2    859
0    430
6    172
7    170
8    170
4    168
1    161
9    134
5    125
3     97
Name: labels, dtype: int64

Еще раз - на обучающей выборке мы сделали апсэмплинг, валидационная оставлена без изменений

Нужно задать веса классов - они будут обратно пропорциональны встречаемости класса в выборке

In [41]:
weight = list(len(train_df['labels']) / (10 * np.bincount(train_df['labels'])))

Воспользуемся библиотекой simple_transformers и дообучим rubert-tiny2 как классификатор.

In [42]:
# зададим конфигурацию модели - вос
model_new = ClassificationModel(
    model_type='bert',
    model_name="cointegrated/rubert-tiny2", 
    tokenizer_name="cointegrated/rubert-tiny2", 
    num_labels=len(train_df['labels'].value_counts()),  # 10 категорий
    weight=weight, # посчитанные выше веса
    use_cuda=False,
    args={"reprocess_input_data": True, 
          "overwrite_output_dir": True, 
          'num_train_epochs': 4, # 4 эпохи
          'manual_seed': 42, 
          'optimizer': 'AdamW',
          'evaluate_during_training': False, 
          'weight_decay': 0,
          'scheduler': 'linear_schedule_with_warmup', 
          'sliding_window': False,
          'config': {"output_hidden_states": True} # делаем возможность использования дообученной модели как энкодера
         }
)

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

Проводим дообучение

In [43]:
%%time
model_new.train_model(train_df)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/2486 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/311 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/311 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/311 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/311 [00:00<?, ?it/s]

CPU times: user 23min 21s, sys: 3min 35s, total: 26min 56s
Wall time: 7min 10s


(1244, 0.8023610526031045)

Оценим результаты дообучения на валидационной выборке

In [44]:
result, model_outputs, wrong_predictions = model_new.eval_model(eval_df)
predictions, raw_outputs, embs, smth_else = model_new.predict(list(eval_df['text']))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/25 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/196 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Running Prediction:   0%|          | 0/25 [00:00<?, ?it/s]

Возьмем предсказания дообученного bert как классификатора и проверим на целевой метрике

In [45]:
def one_label_to_many(df, classes = [x for x in range(17)]):
    y_test = []
    min_class = min(classes)
    count_class = len(classes)

    for ll in df:
        mass = [0]*count_class
        mass[int(ll) - min_class] = 1
        y_test.append(mass)

    return y_test

In [46]:
val_probs = one_label_to_many(predictions, classes = [x for x in range(len(cut_cat))])
y_val_s = y_val.map(cut_cat).astype('int')
display(y_val_s.value_counts())
roc_auc_score(y_val_s, val_probs, multi_class='ovo')

2    95
0    48
9    15
5    14
3    11
6     5
4     3
7     2
1     2
8     1
Name: Категория, dtype: int64

0.8130506189716715

Заметно, что значение целевой метрики на валидации выросло на 5 п.п.

Загрузим дообученную модель, чтобы ей векторизовать обучающую выборку для обучения логистической регрессии

In [47]:
model_ft = RepresentationModel(
        model_type="bert",
        model_name="./outputs",
        use_cuda=False)

Some weights of the model checkpoint at ./outputs were not used when initializing BertForTextRepresentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Векторизуем дообученным bert обучающую выборку и валидационную, выберем cls-эмбеддинг

In [48]:
%%time
ft_train = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(model_ft.encode_sentences(upsed['text'], 
                                                                                  combine_strategy=0)))) # cls

CPU times: user 59.6 s, sys: 3.66 s, total: 1min 3s
Wall time: 15.9 s


In [49]:
%%time
ft_val = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(model_ft.encode_sentences(X_val['text'], 
                                                                                  combine_strategy=0))))

CPU times: user 4.47 s, sys: 284 ms, total: 4.75 s
Wall time: 1.19 s


Обучим логистическую регрессию на этих эмбеддингах

In [50]:
logit.fit(ft_train, y_train)

LogisticRegression(C=1, class_weight='balanced', max_iter=200, random_state=0)

In [51]:
holdout = ft_val

holdout_probs = pd.DataFrame(logit.predict_proba(holdout), columns=logit.classes_)
h_i = holdout_probs.idxmax(axis=1)
        
for col in holdout_probs.columns:
    holdout_probs[col].values[:] = 0
            
for row, idx in zip(holdout_probs.iterrows(), h_i):
    row[1][idx] = 1
    
roc_auc_score(y_val, holdout_probs, multi_class='ovo', labels=logit.classes_)

0.8142202096149465

Качество предсказаний на валидационной выборке также немного выше, чем у модели, использованной как классификатор

## Предсказание теста

Векторизуем дообученным rubert-tiny2 тестовую выборку (только столбец 'text')

А также векторизуем тестовую выборке предобученным rubert-tiny2

In [52]:
%%time
test_df = pd.DataFrame(embed(test, test='test', cls=False))

ft_test = pd.DataFrame(torch.nn.functional.normalize(torch.tensor(model_ft.encode_sentences(test['text'], 
                                                                                  combine_strategy=0))))

CPU times: user 43.9 s, sys: 4.94 s, total: 48.9 s
Wall time: 12.3 s


Получим предсказания для тестовой выборки дообученным bert-ом как классификатором

In [53]:
test_predictions, raw_outputs, _, __ = model_new.predict(list(test['text']))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/125 [00:00<?, ?it/s]

Обучим логистическую регрессию на эмбеддингах дообученного bert

In [54]:
logit_new = LogisticRegression(random_state=2, penalty='l2', C=0.9, class_weight='balanced', max_iter=200)
logit_new.fit(ft_train, y_train)
l_new_preds = pd.Series(logit_new.predict(ft_test))

Чтобы избежать переобучения, обучим k ближайших соседей и метод опороных векторов

Обучим k ближайших соседей на эмбеддингах предобученного bert

In [55]:
knn = KNeighborsClassifier(n_neighbors=9, weights='distance', metric='cosine', algorithm='brute', leaf_size=30)
knn.fit(train, y_train)

KNeighborsClassifier(algorithm='brute', metric='cosine', n_neighbors=9,
                     weights='distance')

Применим метод опороных векторов на эмбеддингах предобученного bert

In [56]:
svm = LinearSVC(loss='hinge', multi_class='ovr', class_weight='balanced',  random_state=42, intercept_scaling=1)

clf = CalibratedClassifierCV(svm) 
clf.fit(train, y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(class_weight='balanced',
                                                loss='hinge', random_state=42))

Далее добавим предсказания этих моделей с небольшими весами в итоговое предсказание

In [57]:
def blend_bert_new_logit(b, k, s):
    
    # получим вероятности для классов дообученным bert-ом как классификатором
    bert_soft_preds = softmax(raw_outputs, axis=1)
    
    # вероятности логистичекой регрессии на дообученных эмбеддингах
    log_probs = logit_new.predict_proba(ft_test)
    # вероятности ближайших соседей на предобученных эмбеддингах
    knn_probs = knn.predict_proba(test_df)
    # вероятности метода опорных векторов на предобученных эмбеддингах
    svm_probs = clf.predict_proba(test_df)
    
    # смешиваем предсказания
    blended = bert_soft_preds * b + log_probs * (1 - b - k - s) + knn_probs * k + svm_probs * s
    blended = pd.DataFrame(blended, columns=logit_new.classes_)
    
    # отбираем категорию с большей вероятностью
    blend = blended.idxmax(axis=1)
    
    # проставляем по словарю значения редких категорий (то, что мы сформировали в начале)
    for idx in hand_cats.keys():
        blend.iloc[idx] = hand_cats[idx]
           
    return blend

Выберем веса для моделей:

    0.25 - дообученный bert как классфикатор
    0.05 - 9 ближайших соседей по косинусному расстоянию предобученных эмбеддингов
    0.1 - метод опорных векторов на предобученных эмбеддингах
    0.6 - логистическая регрессия на эмбеддингах дообученного bert

In [58]:
b_b_n = blend_bert_new_logit(0.25, 0.05, 0.1)

In [59]:
ss = pd.read_csv('sample_solution.csv')
ss['Категория'] = b_b_n
from datetime import datetime
nownow = datetime.now()
filename = nownow.strftime(format='%d_%m_%y_%H_%M_%S')
ss.to_csv(filename + '.csv', index=False)